In [12]:
import sys
import os
sys.path.insert(0, 'D:\\PROGRAMMING\\mlflow')

In [13]:
import mlflow
import mlflow.pytorch
from dataclasses import dataclass, asdict

import torch
import torch.nn as nn
import numpy as np

from time import sleep

print(os.path.dirname(mlflow.__file__))

D:\PROGRAMMING\mlflow\mlflow


In [14]:
@dataclass
class Config:
    EPOCHS: int = 100
    LEARNING_RATE: int = 0.1

CONFIG = Config()

In [15]:
mlflow.set_experiment('Issue #12871')
def mlflow_run_decorator(run_name=None):
    def decorator(func):
        def wrapper(*args, **kwargs):
            mlflow.start_run(run_name=run_name)
            try:
                result = func(*args, **kwargs)
                mlflow.set_tag("Status", "SUCCEESS")
            except Exception as e:
                mlflow.log_param("Exception", e)
                mlflow.set_tag("Status", "FAIL")
                raise e
            finally:
                mlflow.end_run()
            return result
        return wrapper
    return decorator

In [16]:
X = np.random.rand(100, 1)
y = 1 + 2 * X + .1 * np.random.randn(100, 1)

In [17]:
class LinearRegression(nn.Module):
    def __init__(self):
        super(LinearRegression, self).__init__()
        self.linear = nn.Linear(1, 1)

    def forward(self, x):
        return self.linear(x)

In [18]:
model = LinearRegression()
criterion = nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=CONFIG.LEARNING_RATE)

In [19]:
@mlflow_run_decorator(run_name='model save')
def train_model():
    mlflow.log_params(asdict(CONFIG))
    
    for epoch in range(CONFIG.EPOCHS):
        inputs = torch.from_numpy(X.astype(np.float32))
        labels = torch.from_numpy(y.astype(np.float32))

        outputs = model(inputs)
        loss = criterion(outputs, labels)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        mlflow.log_metric('loss', loss.item(), step=epoch)
        
        if epoch % 10 == 0:
            print(f'Epoch {epoch}/{CONFIG.EPOCHS}, Loss: {loss.item()}')
    
        sleep(0)
    
    mlflow.pytorch.log_model(model, "Issue12871_model")

In [20]:
train_model()

Epoch 0/100, Loss: 0.881028413772583
Epoch 10/100, Loss: 0.0946277529001236
Epoch 20/100, Loss: 0.07073307037353516
Epoch 30/100, Loss: 0.05438990518450737
Epoch 40/100, Loss: 0.0423123799264431
Epoch 50/100, Loss: 0.033384863287210464
Epoch 60/100, Loss: 0.02678576298058033
Epoch 70/100, Loss: 0.021907804533839226
Epoch 80/100, Loss: 0.018302064388990402
Epoch 90/100, Loss: 0.01563677377998829


2024/08/04 02:07:40 WARNING mlflow.utils.requirements_utils: Found torch version (2.3.1+cu121) contains a local version label (+cu121). MLflow logged a pip requirement for this package as 'torch==2.3.1' without the local version label to make it installable from PyPI. To specify pip requirements containing local version labels, please use `conda_env` or `pip_requirements`.
2024/08/04 02:07:44 WARNING mlflow.utils.requirements_utils: Found torch version (2.3.1+cu121) contains a local version label (+cu121). MLflow logged a pip requirement for this package as 'torch==2.3.1' without the local version label to make it installable from PyPI. To specify pip requirements containing local version labels, please use `conda_env` or `pip_requirements`.
2024/08/04 02:07:44 WARNING mlflow.models.model: Input example should be provided to infer model signature if the model signature is not provided when logging the model.


In [21]:
model = mlflow.pytorch.load_model('./mlruns/524980164502243114/7d5bf6dfb76f420bb11a5306ad2d2ae9/artifacts/Issue12871_model')

In [22]:
model

LinearRegression(
  (linear): Linear(in_features=1, out_features=1, bias=True)
)